In [1]:
!pip install pennylane-lightning-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 13.1 MB/s eta 0:00:00


In [2]:
!nvidia-smi

Fri Apr 11 10:16:16 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
import torch
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
# ------------------------------
# 1. Load and preprocess the data
# ------------------------------
# Load the UCI Breast Cancer (WDBC) dataset
data = load_breast_cancer()
X = data.data   # 30 features
y = data.target # 0 for malignant, 1 for benign

X_Scaled=StandardScaler().fit_transform(X)
df=pd.DataFrame(X_Scaled,columns=data.feature_names)
# Reduce to 4 features using PCA (to keep the quantum circuit manageable)
pca = PCA(n_components=8)
X_reduced = pca.fit_transform(X_Scaled)

print(pca.explained_variance_ratio_.sum())

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_reduced,y, test_size=0.2, random_state=42)
print(torch.cuda.is_available())
print("Size of Train_set : ",X_train.shape)
print("Size of Test_set : ",X_test.shape)

0.925982538685191
True
Size of Train_set :  (455, 8)
Size of Test_set :  (114, 8)


In [4]:
import torch

print("Total CUDA GPUs PyTorch can see:", torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print(f"Device {i}: {torch.cuda.get_device_name(i)}")

Total CUDA GPUs PyTorch can see: 2
Device 0: Tesla T4
Device 1: Tesla T4


In [5]:
svm = SVC(kernel='linear')
st=time.time()
svm.fit(X_train, y_train)
et=time.time()
y_pred_svm = svm.predict(X_test)
accuracy_svm = np.mean(y_pred_svm == y_test) * 100
print(f"Classical Kernel SVM Accuracy: {accuracy_svm:.2f}%")
print(f"Total Time Taken by Classical Kernel:{et-st}")
print(torch.device)

Classical Kernel SVM Accuracy: 98.25%
Total Time Taken by Classical Kernel:0.006212711334228516
<class 'torch.device'>


In [6]:
import torch

device0 = torch.device("cuda:0")  # First Tesla T4
device1 = torch.device("cuda:1")  # Second Tesla T4

print(torch.cuda.get_device_name(0))  # Should print Tesla T4
print(torch.cuda.get_device_name(1))

Tesla T4
Tesla T4


In [7]:
import pennylane as qml
import multiprocessing as mp
import numpy as np
import os
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

NUM_QUBITS = 8

# Define feature map options
def angle_feature_map(x, wires):
    """Angle Embedding feature map with additional rotations"""
    x = np.pi * (x - min(x)) / (max(x) - min(x) + 1e-6)
    qml.AngleEmbedding(x, wires=wires, rotation='Z')
    for i in range(NUM_QUBITS):
        qml.RY(x[i], wires=i)

def amplitude_feature_map(x, wires):
    """Amplitude Embedding feature map with padding and normalization"""
    x = x / np.sqrt(np.sum(np.abs(x)**2) + 1e-6)
    qml.AmplitudeEmbedding(x, wires=wires, pad_with=0.0, normalize=True)
    for i in range(NUM_QUBITS):
        qml.RZ(weights[i], wires=i)

FEATURE_MAPS = {
    'angle': angle_feature_map,
    'amplitude': amplitude_feature_map
}

def make_device():
    """Create a single GPU device"""
    return qml.device("lightning.gpu", wires=NUM_QUBITS)

def quantum_kernel_element(x1, x2, dev, weights, feature_map_type='angle'):
    feature_map = FEATURE_MAPS[feature_map_type]
    
    @qml.qnode(dev)
    def circuit(weights):
        feature_map(x1, wires=range(NUM_QUBITS))
        qml.adjoint(feature_map)(x2, wires=range(NUM_QUBITS))
        return qml.probs(wires=range(NUM_QUBITS))
    
    return circuit(weights)[0].item()

def compute_kernel_matrix(X1, X2, weights, feature_map_type='angle'):
    """Compute kernel matrix using a single GPU"""
    dev = make_device()
    n1, n2 = len(X1), len(X2)
    K = np.zeros((n1, n2))
    
    for i in range(n1):
        for j in range(n2):
            K[i, j] = quantum_kernel_element(X1[i], X2[j], dev, weights, feature_map_type)
    
    return K


feature_map_type = 'angle'
weights = np.random.uniform(low=0, high=2*np.pi, size=(NUM_QUBITS,))

print("Computing training kernel matrix...")
K_train = compute_kernel_matrix(X_train, X_train, weights, feature_map_type)
print("Computing test kernel matrix...")
K_test = compute_kernel_matrix(X_test, X_train, weights, feature_map_type)

# clf = SVC(kernel='precomputed', C=1.0, class_weight='balanced')
clf = SVC(kernel='precomputed', C=1.0, class_weight='balanced')
clf.fit(K_train, y_train)

train_acc = accuracy_score(y_train, clf.predict(K_train))
test_acc = accuracy_score(y_test, clf.predict(K_test))
    
print(f"Training accuracy with {feature_map_type} feature map: {train_acc:.4f}")
print(f"Test accuracy with {feature_map_type} feature map: {test_acc:.4f}")

Computing training kernel matrix...
Computing test kernel matrix...
Training accuracy with angle feature map: 0.9912
Test accuracy with angle feature map: 0.9561


In [8]:
clf = SVC(kernel='precomputed')
clf.fit(K_train, y_train)

train_acc = accuracy_score(y_train, clf.predict(K_train))
test_acc = accuracy_score(y_test, clf.predict(K_test))
    
print(f"Training accuracy with {feature_map_type} feature map: {train_acc:.4f}")
print(f"Test accuracy with {feature_map_type} feature map: {test_acc:.4f}")

Training accuracy with angle feature map: 0.9912
Test accuracy with angle feature map: 0.9561


In [9]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, K_train, y_train, cv=5)
print(f"Cross-validation scores: {scores.mean():.4f} ± {scores.std():.4f}")

Cross-validation scores: 0.9582 ± 0.0264


In [10]:
clf_classical = SVC(kernel='rbf', C=1.0, class_weight='balanced')
clf_classical.fit(X_train, y_train)
print(f"Classical RBF Test Accuracy: {clf_classical.score(X_test, y_test):.4f}")

Classical RBF Test Accuracy: 0.9561
